In [1]:
from keras.models import load_model
from PIL import Image
from numpy import expand_dims
from numpy import asarray

import numpy as np
import pickle
import cv2

In [6]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = load_model("../Models/FaceNet/facenet_keras.h5")

myfile = open("../Data/signature.pkl", "rb")
db = pickle.load(myfile)
myfile.close()

In [3]:
db

{'yasir': array([[ 1.1472378 , -0.14560148, -1.2207143 ,  0.86740947, -0.6584511 ,
          1.7314054 ,  0.26071918, -0.7417967 , -0.38172582,  1.6192524 ,
         -0.6029541 , -1.3463066 ,  0.21167357, -1.5171381 , -0.8065996 ,
         -1.7359992 ,  0.7274879 , -0.9320656 ,  0.91032887, -1.376366  ,
          0.8205123 , -0.58977664,  0.07671417, -0.57645863,  0.19138135,
          0.57225764, -0.22292209,  0.8103644 , -0.46793556,  0.30820343,
          0.6648637 ,  0.8360966 ,  1.5616367 , -1.1568385 ,  1.5107288 ,
         -0.92915004, -0.9427135 ,  0.17257035,  1.3283756 ,  0.09193929,
         -0.15075485,  0.2796951 , -0.8401743 , -0.64400613, -0.37222382,
         -0.34793392, -0.84640706,  2.4472456 , -0.526461  ,  0.29501292,
          1.3032044 , -0.5009302 ,  0.13624032,  0.43029135,  2.0582535 ,
          1.7739886 ,  0.46875495, -1.6923292 ,  0.29209757, -1.8634319 ,
         -1.4435849 , -3.2254581 ,  1.8089039 , -1.5726203 , -0.44333985,
         -1.6616787 , -1.9352

In [7]:
cap = cv2.VideoCapture(2)

while(1):
    _, imgVideo = cap.read()
    
    FaceDetect = HaarCascade.detectMultiScale(imgVideo, 1.3, 10)
    
    for (x1, y1, width, height) in FaceDetect:
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        
        img = cv2.cvtColor(imgVideo, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_array = asarray(img)
        
        face = img_array[y1:y2, x1:x2]
        
        face = Image.fromarray(face)
        face = face.resize((160,160))
        face = asarray(face)
        
        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        
        face = expand_dims(face, axis=0)
        signature = MyFaceNet.predict(face)
        # print(signature)
        
        min_dist = 100
        identify = ''
        for key, value in db.items():
            dist = np.linalg.norm(value-signature)
            # print(dist)
            if (dist < min_dist):
                min_dist = dist 
                if min_dist > 7:
                    identify = "Unknown"
                else :
                    identify  = key
                    print(identify,",",min_dist)
                
        cv2.rectangle(imgVideo, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.rectangle(imgVideo, (x1,y1-40), (x2,y1), (0,255,0), -2)
        cv2.putText(imgVideo, identify + ", " + str(round(min_dist, 2)), (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)

    cv2.imshow('Face Recognition', imgVideo)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 50ms/step
18101152630087 , 6.5480537
1/1 [==============================] - 0s 55ms/step
18101152630087 , 6.2334437
1/1 [==============================] - 0s 52ms/step
18101152630087 , 6.6292396
1/1 [==============================] - 0s 92ms/step
18101152630087 , 6.2449846
1/1 [==============================] - 0s 66ms/step
18101152630087 , 6.0944304
1/1 [==============================] - 0s 71ms/step
18101152630087 , 6.7382436
1/1 [==============================] - 0s 57ms/step
18101152630087 , 6.566847
1/1 [==============================] - 0s 53ms/step
18101152630087 , 6.8650794
1/1 [==============================] - 0s 55ms/step
18101152630087 , 6.801339
1/1 [==============================] - 0s 52ms/step
18101152630087 , 6.247364
1/1 [==============================] - 0s 54ms/step
18101152630087 , 6.5657926
1/1 [==============================] - 0s 55ms/step
18101152630087 , 6.0525985
1/1 [==============================] - 0s 55ms/step
181